In [ ]:
import glob
from enrich.tei import teis_to_traindata
from enrich.spacy_utils.data_prep import clean_train_data, traindata_to_csv, csv_to_traindata

In [ ]:
# source_dir = r"C:\Users\pandorfer\Documents\Redmine\konde\schnitzler-tagebuch\data\editions\*.xml"
# source_dir = r"C:\Users\pandorfer\Documents\Redmine\konde\thun-project\thun-data\editions\*.xml"
source_dir = r"C:\Users\pandorfer\Documents\Redmine\konde\Hermann-Bahr_Arthur-Schnitzler\app\data\letters\**\*.xml"

In [ ]:
files = [x for x in glob.iglob(source_dir, recursive=False)]
len(files)

In [ ]:
# generate train data
TRAIN_DATA = teis_to_traindata(files, ne_xpath='.//tei:body//tei:*[@key]', verbose=False) #habs
# TRAIN_DATA = teis_to_traindata(files, start_node='.//tei:div[@type="transcript"]', verbose=False) #thun
len(TRAIN_DATA)

In [ ]:
# clean train data (removing 'empty samples' and samples with less then 4 entities)
CLEAN_TRAIN_DATA = clean_train_data(TRAIN_DATA, 2)
len(CLEAN_TRAIN_DATA)

In [ ]:
# save cleaned train data as csv
out = traindata_to_csv(CLEAN_TRAIN_DATA, 'data/hbas.csv')

In [ ]:
# load saved train data as TRAIN_DATA list of lists
loaded_train_data = csv_to_traindata(out)

In [ ]:
##### compare models

In [ ]:
import spacy

In [ ]:
data = csv_to_traindata('data/hbas.csv')

In [ ]:
len(data)

In [ ]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for x in examples:
        doc_gold_text = ner_model.make_doc(x[0])
        gold = GoldParse(doc_gold_text, entities=x[1]['entities'])
        pred_value = ner_model(x[0])
        scorer.score(pred_value, gold)
    return scorer.scores

In [ ]:
examples = data[800:len(data)]

In [ ]:
for x in ['de_core_news_sm', 'data/thun', 'data/stb', 'data/hbas', 'data/hbas_new', 'data/hbas_new_100']:
    ner_model = spacy.load(x)
    results = evaluate(ner_model, examples)
    print(x, "p: {}; f: {}; r: {}".format(results['ents_p'], results['ents_f'], results['ents_r']))